# 模型参数学习方法总结

- <a href='#gradient'>梯度下降法</a>
- <a href='#min_square'>最小二乘法</a>
- <a href='#newton'>牛顿法</a>
- <a href='#newton2'>拟牛顿法</a>

---

# <a name='gradient'>梯度下降法</a>

在求解机器学习算法的模型参数，即无约束优化问题时，**梯度下降（Gradient Descent）**是最常采用的方法之一，另一种常用的方法是**最小二乘法**。这里就对梯度下降法做一个完整的总结。

## 1.梯度

在微积分里面，**对多元函数的参数求∂偏导数，把求得的各个参数的偏导数以向量的形式写出来，就是梯度。**

比如：  
函数f(x,y), 分别对x,y求偏导数，求得的梯度向量就是$(∂f/∂x, ∂f/∂y)^T$,简称$grad\ f(x,y)$或者$▽f(x,y)$。

对于在点$(x_0,y_0)$的具体梯度向量就是$(∂f/∂x_0, ∂f/∂y_0)^T$.或者$▽f(x_0,y_0)$，如果是3个参数的向量梯度，就是$(∂f/∂x, ∂f/∂y，∂f/∂z)^T$,以此类推。

那么这个**梯度向量求出来有什么意义**呢？

他的意义：  
从几何意义上讲，就是**函数变化增加最快的地方**。
具体来说，对于函数f(x,y),在点$(x_0,y_0)$，沿着梯度向量的方向就是$(∂f/∂x_0, ∂f/∂y_0)^T$的方向是f(x,y)增加最快的地方。   
或者说，**沿着梯度向量的方向，更加容易找到函数的最大值**。   
反过来说，**沿着梯度向量相反的方向，也就是 $-(∂f/∂x_0, ∂f/∂y_0)^T$的方向，梯度减少最快，也就是更加容易找到函数的最小值。**



## 2. 梯度下降与梯度上升

在机器学习算法中，在**最小化损失函数**时，可以通过**梯度下降法**来一步步的迭代求解，得到最小化的损失函数，和模型参数值。

反过来，如果我们需要**求解损失函数的最大值**，这时就需要用**梯度上升法**来迭代了。

**梯度下降法和梯度上升法是可以互相转化的。**    
比如：  
我们需要求解损失函数f(θ)的最小值，这时我们需要用梯度下降法来迭代求解。但是实际上，我们可以反过来求解损失函数 -f(θ)的最大值，这时梯度上升法就派上用场了。

下面来详细总结下梯度下降法。        



## 3. 梯度下降法算法解释

### 3.1 梯度下降的直观解释

首先来看看梯度下降的一个直观的解释。

比如：  
我们在一座大山上的某处位置，由于我们不知道怎么下山，于是决定走一步算一步，也就是在每走到一个位置的时候，求解当前位置的梯度，沿着梯度的负方向，也就是当前最陡峭的位置向下走一步，然后继续求解当前位置梯度，向这一步所在位置沿着最陡峭最易下山的位置走一步。这样一步步的走下去，一直走到觉得我们已经到了山脚。当然这样走下去，有可能我们不能走到山脚，而是到了某一个局部的山峰低处。



从上面的解释可以看出，**梯度下降不一定能够找到全局的最优解，有可能是一个局部最优解。**

当然，**如果损失函数是凸函数，梯度下降法得到的解就一定是全局最优解**。

<img src='./images/gradient_descent.png' width='70%'/>


### 3.2 梯度下降的相关概念

在详细了解梯度下降的算法之前，我们先看看相关的一些概念。

1. 步长（Learning rate）：  
**步长决定了在梯度下降迭代的过程中，每一步沿梯度负方向前进的长度。**
用上面下山的例子，步长就是在当前这一步所在位置沿着最陡峭最易下山的位置走的那一步的长度。

2. 特征（feature）：   
指的是样本中输入部分，比如2个单特征的样本（$x^{(0)},y^{(0)}）$,$（x^{(1)},y^{(1)}）$,则第一个样本特征为$x^{(0)}$，第一个样本输出为$y^{(0)}$。

3. 假设函数（hypothesis function）：   
在监督学习中，为了拟合输入样本，而使用的假设函数，记为$h_θ(x)$。比如对于单个特征的m个样本$（x^{(i)},y^{(i)}）$(i=1,2,...m),可以采用拟合函数如下： $$h_θ(x)=θ_0+θ_1x$$

4. 损失函数（loss function）：  
为了**评估模型拟合的好坏**，通常用**损失函数来度量拟合的程度**。损失函数极小化，意味着拟合程度最好，对应的模型参数即为最优参数。

在线性回归中，损失函数通常为样本输出和假设函数的差取平方。比如对于m个样本$(x+i,y_i）$(i=1,2,...m),采用线性回归，损失函数为：
$$J(θ_0,θ_1)=∑_{i=1}^m(h_θ(x_i)−y_i)^2$$

其中，$x_i$表示第$i$个样本特征，$y_i$表示第$i$个样本对应的输出，$h_θ(x_i)$为假设函数。   

### 3.3 梯度下降法的详细算法

梯度下降法的算法可以有**代数法**和**矩阵法（也称向量法）**两种表示，如果对矩阵分析不熟悉，则代数法更加容易理解。

不过矩阵法更加的简洁，且由于使用了矩阵，实现逻辑更加的一目了然。

这里先介绍代数法，后介绍矩阵法。

#### 3.3.1 梯度下降法的代数方式描述

1. 先决条件： 确认优化模型的假设函数和损失函数。

比如：  
对于线性回归，假设函数表示为 $$h_θ(x_1,x_2,...x_n)=θ_0+θ_1x_1+...+θ_nx_n$$其中$θ_i (i = 0,1,2... n)$为模型参数，$x_i (i = 0,1,2... n)$为每个样本的n个特征值。  

这个表示可以简化，我们增加一个特征$x_0=1 $，这样$$h_θ(x_0,x_1,...x_n)=∑_{i=0}^nθ_ix_i$$

同样是线性回归，对应于上面的假设函数，损失函数为：

$$J(θ_0,θ_1...,θ_n)=\frac{1}{2m}∑_{j=0}^m(h_θ(x^{(j)}_0,x^{(j)}_1,...x^{(j)}_n)−y_j)^2$$

2. 算法相关参数初始化：  
主要是初始化$θ_0$,$θ_1$,...,$θ_n$，算法终止距离$ε$以及步长$α$。  
在没有任何先验知识的时候，我喜欢将所有的θ初始化为0，将步长初始化为1。  
在调优的时候再 优化。  

3. 算法过程：

1）确定当前位置的损失函数的梯度，对于$θ_i$,其梯度表达式如下：

$$\frac{∂}{∂θ_i}J(θ_0,θ_1...,θ_n)$$

2）用**步长**乘以**损失函数的梯度**，得到当前位置下降的距离，即$α\frac{∂}{∂θ_i}J(θ_0,θ_1...,θ_n)$对应于前面登山例子中的某一步。

3）确定是否**所有的$θ_i$，梯度下降的距离都小于ε，如果小于ε则算法终止**，当前所有的$θ_i(i=0,1,...n)$即为最终结果。否则进入步骤4.

4）更新所有的θ，对于$θ_i$，其更新表达式如下。更新完毕后继续转入步骤1.

$$θ_i=θ_i−α\frac{∂}{∂θ_i}J(θ_0,θ_1...,θ_n)$$
$$新θ = 现在的θ- 步长（learning\ rate) * 损失函数的梯度$$

下面用**线性回归的例子来具体描述梯度下降**。

假设我们的样本是$(x^{(0)}_1,x^{(0)}_2,...x^{(0)}_n,y_0),(x^{(1)}_1,x^{(1)}_2,...x^{(1)}_n,y_1),...(x^{(m)}_1,x^{(m)}_2,...x^{(m)}_n,y_m)$,   
损失函数如前面先决条件所述：
$$J(θ_0,θ_1...,θ_n)=\frac{1}{2m}∑_{j=0}^m(h_θ(x^{(j)}_0,x^{(j)}_1,...x^{(j)}_n)−y_j)^2$$

则在算法过程步骤1中对于$θ_i$ 的偏导数计算如下： 　
$$\frac{∂}{∂θ_i}J(θ_0,θ_1...,θ_n)=\frac{1}{m}∑_{j=0}^m(h_θ(x^{(j)}_0,x^{(j)}_1,...x^{(j)}_n)−y_j)x^{(j)}_i$$

由于样本中没有$x_0$，所以，令上式中所以的$x_0^j$为1。

步骤4中θi的更新表达式如下：
$$θ_i=θ_i−α\frac{1}{m}∑_{j=0}^m(h_θ(x^{(j)}_0,x^{(j)}_1,...x^j_n)−y_j)x^{(j)}_i$$

从这个例子可以看出：  
**当前点的梯度方向是由所有的样本决定的**，加$\frac{1}{m}$是为了好理解。由于步长也为常数，他们的乘机也为常数，所以这里$α\frac{1}{m}$可以用一个常数表示。

在下面第4节会详细讲到的梯度下降法的变种，他们主要的区别就是对样本的采用方法不同。这里我们采用的是用**所有样本**。



#### 3.3.2 梯度下降法的矩阵方式描述


这一部分主要讲解梯度下降法的矩阵方式表述，相对于3.3.1的代数法，要求有一定的矩阵分析的基础知识，尤其是矩阵求导的知识。

1. 先决条件： 
和3.3.1类似， 需要确认优化模型的**假设函数**和**损失函数**。

对于线性回归，假设函数$h_θ(x_1,x_2,...x_n)=θ_0+θ_1x_1+...+θ_nx_n$的矩阵表达方式为：
$$h_θ(x)=Xθ$$

其中， 假设函数$h_θ(X)$为m x 1的向量，θ为(n+1) x 1的向量，里面有n个代数法的模型参数。X为m x (n+1)维的矩阵。m代表样本的个数，n+1代表样本的特征数。

$$h_θ(x)_{m*1}=X_{m*(n+1)}θ_{(n+1)*1}$$

损失函数的表达式为：
 $$J(θ)=\frac{1}{2}(Xθ−Y)^T(Xθ−Y)$$ 
其中Y是样本的输出向量，维度为m x 1.
  $$J(θ)=\frac{1}{2}(X_{m*(n+1)}θ_{(n+1)*1}−Y_{m*1})^T_{1*m}(X_{m*(n+1)}θ_{(n+1)*1}−Y_{m*1})_{m*1}$$ 
  
这样得到的损失函数结果J(θ)是一个标量。

2. 算法相关参数初始化:    
θ向量可以初始化为默认值，或者调优后的值。   
算法终止距离ε，步长α和3.3.1比没有变化。

3. 算法过程：

1）确定当前位置的损失函数的梯度，对于θ向量,其梯度表达式如下：
$$\frac{∂}{∂θ}J(θ)$$

2）用步长乘以损失函数的梯度，得到当前位置下降的距离，即$α\frac{∂}{∂θ}J(θ)$对应于前面登山例子中的某一步。

3）确定θ向量里面的每个值,梯度下降的距离都小于ε，如果小于ε则算法终止，当前θ向量即为最终结果。否则进入步骤4.

4）更新θ向量，其更新表达式如下。更新完毕后继续转入步骤1.
$$θ=θ−α\frac{∂}{∂θ}J(θ)$$

还是用线性回归的例子来描述具体的算法过程。

损失函数对于θ向量的偏导数计算如下：
$$\frac{∂}{∂θ}J(θ)=X^T(Xθ−Y)$$

步骤4中θ向量的更新表达式如下：
$$θ=θ−αX^T(Xθ−Y)$$

对于3.3.1的代数法，可以看到矩阵法要简洁很多。这里面用到了**矩阵求导链式法则**，和两个矩阵求导的公式。

公式1：$$\frac{∂}{∂X}(XX^T)=2X$$
公式2：$$\frac{∂}{∂θ}(Xθ)=X^T$$

如果需要熟悉矩阵求导建议参考张贤达的《矩阵分析与应用》一书。

这个线性回归的例子中的损失函数的表达式为：
 $$J(θ)=\frac{1}{2}(Xθ−Y)^T(Xθ−Y)$$ 
根据**矩阵求导链式法则**对上式求导：
 $$\frac{∂}{∂X}J(θ)=\frac{1}{2}(Xθ−Y)^T(Xθ−Y)$$  
 形如$XX^T$，得到$=\frac{1}{2} * 2 (Xθ−Y)$  
 再对$Xθ$求导，得到$=\frac{1}{2} * 2(Xθ−Y) * X^T$
 $$=X^T(Xθ−Y)$$
 


### 3.4 梯度下降的算法调优

在使用梯度下降时，需要进行调优。哪些地方需要调优呢？

1. 算法的步长选择。  
在前面的算法描述中，我提到取步长为1，但是实际上取值取决于数据样本，可以多取一些值，从大到小，分别运行算法，看看迭代效果，**如果损失函数在变小，说明取值有效，否则要增大步长。**   
前面说了。步长太大，会导致迭代过快，甚至有可能错过最优解。步长太小，迭代速度太慢，很长时间算法都不能结束。所以算法的步长需要多次运行后才能得到一个较为优的值。

2. 算法参数的初始值选择。  
初始值不同，获得的最小值也有可能不同，因此梯度下降求得的只是局部最小值；当然如果损失函数是凸函数则一定是最优解。由于**有局部最优解的风险，需要多次用不同初始值运行算法，关键损失函数的最小值，选择损失函数最小化的初值**。

3. 归一化。   
由于样本不同特征的取值范围不一样，可能导致迭代很慢，为了减少特征取值的影响，可以对特征数据归一化，也就是对于每个特征x，求出它的期望$\overline x$和标准差std(x)，然后转化为：
$$\frac{x−\overline x}{std(x)}$$

这样特征的新期望为0，新方差为1，迭代次数可以大大加快。



## 4. 梯度下降法大家族（BGD,SGD,MBGD)

### 4.1 批量梯度下降法 Batch Gradient Descent

批量梯度下降法，是梯度下降法**最常用的形式**，具体做法也就是**在更新参数时使用所有的样本来进行更新**，这个方法对应于前面3.3.1的线性回归的梯度下降算法，也就是说3.3.1的梯度下降算法就是批量梯度下降法。　　

$$θ_i=θ_i−α∑_{j=0}^m(h_θ(x^{(j)}_0,x^{(j)}_1,...x^{(j)}_n)−y_j)x^{(j)}_i$$

由于我们有m个样本，这里求梯度的时候就用了所有m个样本的梯度数据。

### 4.2 随机梯度下降法 Stochastic Gradient Descent

随机梯度下降法，其实和批量梯度下降法原理类似，区别在与求梯度时没有用所有的m个样本的数据，而是仅仅选取**一个样本j**来求梯度。  

对应的更新公式是：
$$θ_i=θ_i−α(h_θ(x^{(j)}_0,x^{(j)}_1,...x^{(j)}_n)−y_j)x^{(j)}_i$$

随机梯度下降法，和4.1的批量梯度下降法是两个极端：
- 一个采用所有数据来梯度下降，
- 一个用一个样本来梯度下降。

自然各自的优缺点都非常突出。
- 对于训练速度来说，
    - 随机梯度下降法由于每次仅仅采用一个样本来迭代，训练速度很快，
    - 而批量梯度下降法在样本量很大的时候，训练速度不能让人满意。
- 对于准确度来说，
    - 随机梯度下降法用于仅仅用一个样本决定梯度方向，导致解很有可能不是最优。
- 对于收敛速度来说，
    - 由于随机梯度下降法一次迭代一个样本，导致迭代方向变化很大，不能很快的收敛到局部最优解。

那么，有没有一个中庸的办法能够结合两种方法的优点呢？  
有！这就是4.3的小批量梯度下降法。

### 4.3 小批量梯度下降法Mini-batch Gradient Descent

小批量梯度下降法是批量梯度下降法和随机梯度下降法的折衷，也就是对于m个样本，我们采用x个样子来迭代，1<x<m。  
一般可以取x=10，当然根据样本的数据，可以调整这个x的值。

对应的更新公式是：

$$θ_i=θ_i−α∑_{j=t}^{t+x-1}(h_θ(x^{(j)}_0,x^{(j)}_1,...x^{(j)}_n)−y_j)x^{(j)}_i$$

## 5. 梯度下降法和其他无约束优化算法的比较

在机器学习中的无约束优化算法，除了梯度下降以外，还有前面提到的最小二乘法，此外还有牛顿法和拟牛顿法。

### 5.1 梯度下降法和最小二乘法
梯度下降法和最小二乘法相比，
- 梯度下降法需要选择**步长**，而最小二乘法不需要。
- 梯度下降法是**迭代求解**，最小二乘法是**计算解析解**。  
如果样本量不算很大，且存在解析解，最小二乘法比起梯度下降法要有优势，计算速度很快。但是如果样本量很大，用最小二乘法由于需要求一个超级大的逆矩阵，这时就很难或者很慢才能求解解析解了，使用迭代的梯度下降法比较有优势。


### 5.2 梯度下降法和牛顿法/逆牛顿法
梯度下降法和牛顿法/拟牛顿法相比，
- 两者都是**迭代求解**，不过梯度下降法是梯度求解，而牛顿法/拟牛顿法是用**二阶的海森矩阵的逆矩阵或伪逆矩阵求解**。
- 相对而言，使用牛顿法/拟牛顿法**收敛更快**。但是每次**迭代的时间比梯度下降法长**。

---

# <a name='min_square'>最小二乘法</a>

最小二乘法是用来做函数拟合或者求函数极值的方法。

在机器学习，尤其是**回归**模型中，经常可以看到最小二乘法的身影，这里就对我对最小二乘法的认知做一个小结。

## 1. 最小二乘法的原理与要解决的问题

最小二乘法是由勒让德在19世纪发现的，原理的一般形式很简单，当然发现的过程是非常艰难的。

形式如下式：
$$目标函数 = Σ(观测值-理论值)^2$$


观测值就是我们的多组样本，理论值就是我们的假设拟合函数。

**目标函数**也就是在机器学习中常说的**损失函数**，我们的目标是得到**使目标函数最小化时候的拟合函数的模型。**


举一个最简单的线性回归的简单例子，比如我们有m个只有一个特征的样本：
$$(x^{(1)},y^{(1)}),(x^{(2)},y^{(2)},...(x^{(m)},y^{(m)})$$

样本采用下面的拟合函数：
$$h_θ(x)=θ_0+θ_1x$$

这样我们的样本有一个特征x，对应的拟合函数有两个参数$θ_0$和$θ_1$需要求出。

我们的目标函数为：

$$J(θ_0,θ_1)=∑_{i=1}^m(y^{(i)}−h_θ(x^{(i)})^2=∑_{i=1}^m(y^{(i)}−θ_0−θ_1x^{(i)})^2$$　

用最小二乘法做什么呢，使$J(θ_0,θ_1)$最小，求出使$J(θ_0,θ_1)$最小时的$θ_0$和$θ_1$，这样拟合函数就得出了。



那么，最小二乘法怎么才能使J(θ0,θ1)最小呢？

## 2. 最小二乘法的代数法解法

上面提到要使$J(θ_0,θ_1)$最小，方法就是对$θ_0$和$θ_1$分别来求偏导数，令偏导数为0，得到一个关于$θ_0$和$θ_1$的二元方程组。

求解这个二元方程组，就可以得到$θ_0$和$θ_1$的值。

下面我们具体看看过程。

$J(θ_0,θ_1)$对$θ_0$求导，得到如下方程：

$$∑_{i=1}^m(y^{(i)}−θ_0−θ_1x^{(i)})=0$$               

$J(θ_0,θ_1)$对$θ_1$求导，得到如下方程：

$$∑_{i=1}^m(y^{(i)}−θ_0−θ_1x^{(i)})x^{(i)}=0$$

上面对$\theta_0$和$\theta_1$求导的两式组合成一个二元一次方程组，容易求出$\theta_0$和$\theta_1$的值：

<img src='./images/min_square.png' width='70%'/>

这个方法很容易推广到多个样本特征的线性拟合。

拟合函数表示为 $h_θ(x_1,x_2,...x_n)=θ_0+θ_1x_1+...+θ_nx_n$, 其中θi (i = 0,1,2... n)为模型参数，$x_i (i = 0,1,2... n)$为每个样本的n个特征值。这个表示可以简化，我们增加一个特征$x_0=1$ ，这样拟合函数表示为：
$$h_θ(x_0,x_1,...x_n)=∑_{i=0}^nθ_ix_i$$

损失函数表示为：

$$J(θ_0,θ_1...,θ_n)=∑_{j=1}^m(h_θ(x^{(j)}_0),x^{(j)}_1,...x^{(j)}_n))−y^{(j)}))^2=∑_{j=1}^m(∑_{i=0}^nθ_ix^{(j)}_i−y^{(j)})^2$$

利用损失函数分别对θi(i=0,1,...n)求导，并令导数为0可得：

<img src='./images/multi_min_square.png' width='50%'/>

这样我们得到一个N+1元一次方程组，这个方程组有N+1个方程，求解这个方程，就可以得到所有的N+1个未知的θ。



这个方法很容易推广到多个样本特征的非线性拟合。原理和上面的一样，都是用损失函数对各个参数求导取0，然后求解方程组得到参数值。这里就不累述了。



## 3. 最小二乘法的矩阵法求解

矩阵法比代数法要简洁，且矩阵运算可以取代循环，所以现在很多书和机器学习库都是用的矩阵法来做最小二乘法。

这里用上面的多元线性回归例子来描述矩阵法解法。

假设函数$h_θ(x_1,x_2,...x_n)=θ_0+θ_1x_1+...+θ_{n−1}x_{n−1}$的矩阵表达方式为：
$$h_θ(x)=Xθ$$

其中，假设函数$h_θ(X)$为mx1的向量，θ为nx1的向量，里面有n个代数法的模型参数。X为mxn维的矩阵。m代表样本的个数，n代表样本的特征数。

损失函数定义为$$J(θ)=\frac{1}{2}(Xθ−Y)^T(Xθ−Y)$$

其中Y是样本的输出向量，维度为m x 1。  
$\frac{1}{2}$在这主要是为了求导后系数为1，方便计算。

根据最小二乘法的原理，我们要对这个损失函数对θ向量求导取0。

结果如下式：

$$\frac{∂}{∂θ}J(θ)=X^T(Xθ−Y)=0$$

对上述求导等式整理后可得：

$$X^TXθ=X^TY$$


两边同时左乘(XTX)−1可得：

$$θ=(X^TX)^{−1}X^TY$$

这样我们就一下子求出了θ向量表达式的公式，免去了代数法一个个去求导的麻烦。只要给了数据，我们就可以用$θ=(X^TX)^{−1}X^TY$算出θ。



## 4. 最小二乘法的局限性和适用场景

从上面可以看出，最小二乘法适用简洁高效，比梯度下降这样的迭代法似乎方便很多。但是这里我们就聊聊最小二乘法的局限性。



首先，最小二乘法需要计算$X^TX$的逆矩阵，有可能它的逆矩阵$(X^TX)^{-1}$不存在，这样就没有办法直接用最小二乘法了，此时梯度下降法仍然可以使用。当然，我们可以通过对样本数据进行整理，去掉冗余特征。让$X^TX$的行列式不为0，然后继续使用最小二乘法。

第二，当样本特征n非常的大的时候，计算$X^TX$的逆矩阵是一个非常耗时的工作（nxn的矩阵求逆），甚至不可行。  
此时以梯度下降为代表的迭代法仍然可以使用。

那这个**n到底多大就不适合最小二乘法呢？**   
如果你没有很多的分布式大数据计算资源，建议**超过10000个特征就用迭代法**吧。   
**或者通过主成分分析降低特征的维度后再用最小二乘法**。

第三，如果拟合函数**不是线性的**，这时无法使用最小二乘法，**需要通过一些技巧转化为线性才能使用**，此时梯度下降仍然可以用。

第四，讲一些特殊情况。  
当样本量m很少，小于特征数n的时候，这时拟合方程是欠定的，常用的优化方法都无法去拟合数据。   
当样本量m等于特征数n的时候，用方程组求解就可以了。   
**当m大于n时，拟合方程是超定的，也就是我们常用与最小二乘法的场景了。**

---

# <a name='newton'>牛顿法</a> Newton's method 

牛顿法是一种在**实数域**和**复数域**上近似求解方程的方法。

方法**使用函数f (x)的泰勒级数的前面几项**来寻找方程f (x) = 0的根。

牛顿法最大的特点就在于它的**收敛速度很快**。

## 1. 牛顿法的具体步骤

首先，选择一个接近函数 f (x)零点的$x_0$，计算相应的 $f (x_0)$ 和切线斜率$f  ' (x_0)$（这里f ' 表示函数 f  的导数）。

然后我们计算穿过点$(x_0,  f  (x_0))$ 并且斜率为$f '(x_0)$的直线和 x 轴的交点的x坐标，也就是求如下方程的解：
$$x f'(x_0) + f(x_0) - x_0 f'(x_0) = 0$$

我们将新求得的点的 x 坐标命名为$x_1$，通常$x_1$会比$x_0$更接近方程f  (x) = 0的解。因此我们现在可以利用$x_1$开始下一轮迭代。

迭代公式可化简为如下所示：
$$x_{n+1} = x_n -\frac{f(x_n)}{f'(x_n)}$$

已经证明，如果f  ' 是连续的，并且待求的零点x是孤立的，那么在零点x周围存在一个区域，只要初始值$x_0$位于这个邻近区域内，那么牛顿法必定收敛。 

并且，如果f  ' (x)不为0, 那么牛顿法将具有平方收敛的性能.   
粗略的说，这意味着每迭代一次，牛顿法结果的有效数字将增加一倍。

下图为一个牛顿法执行过程的例子。

由于牛顿法是基于当前位置的切线来确定下一次的位置，所以牛顿法又被很形象地称为是"切线法"。

牛顿法的搜索路径（二维情况）如下图所示：

牛顿法搜索动态示例图：

<img src='./images/newton.gif' width='70%'/>

## 2. 牛顿法和梯度下降法的效率对比

从本质上去看，牛顿法是二阶收敛，梯度下降是一阶收敛，所以牛顿法就更快。

如果更通俗地说的话，比如你想找一条最短的路径走到一个盆地的最底部，梯度下降法每次只从你当前所处位置选一个坡度最大的方向走一步，牛顿法在选择方向时，不仅会考虑坡度是否够大，还会考虑你走了一步之后，坡度是否会变得更大。所以，可以说牛顿法比梯度下降法看得更远一点，能更快地走到最底部。（牛顿法目光更加长远，所以少走弯路；相对而言，梯度下降法只考虑了局部的最优，没有全局思想。）

根据wiki上的解释，从几何上说，牛顿法就是用一个二次曲面去拟合你当前所处位置的局部曲面，而梯度下降法是用一个平面去拟合当前的局部曲面，通常情况下，二次曲面的拟合会比平面更好，所以牛顿法选择的下降路径会更符合真实的最优下降路径。


<img src='./images/newton_method.png' width='70%'/>

## 3. 牛顿法的优缺点总结
牛顿法的优缺点总结：
- 优点：二阶收敛，收敛速度快；
- 缺点：牛顿法是一种迭代算法，每一步都需要求解目标函数的Hessian矩阵的逆矩阵，计算比较复杂。

---

# <a name='newton2'>拟牛顿法</a> Quasi-Newton Methods

拟牛顿法是求解**非线性优化问题**最有效的方法之一，于20世纪50年代由美国Argonne国家实验室的物理学家W.C.Davidon所提出来。Davidon设计的这种算法在当时看来是非线性优化领域最具创造性的发明之一。不久R. Fletcher和M. J. D. Powell证实了这种新的算法远比其他方法快速和可靠，使得非线性优化这门学科在一夜之间突飞猛进。

拟牛顿法的本质思想是**改善牛顿法每次需要求解复杂的Hessian矩阵的逆矩阵的缺陷**，它使用**正定矩阵来近似Hessian矩阵的逆**，从而简化了运算的复杂度。

拟牛顿法和最速下降法一样只要求每一步迭代时知道目标函数的梯度。

通过测量梯度的变化，构造一个目标函数的模型使之足以产生超线性收敛性。这类方法大大优于最速下降法，尤其对于困难的问题。另外，因为**拟牛顿法不需要二阶导数的信息**，所以有时比牛顿法更为有效。如今，**优化软件中包含了大量的拟牛顿算法用来解决无约束，约束，和大规模的优化问题。**

具体步骤：

拟牛顿法的基本思想如下。

首先构造目标函数在当前迭代$x_k$的二次模型：

<img src='./images/quasi_newton.png' width='70%'/>

这里$B_k$是一个对称正定矩阵，于是我们取这个二次模型的最优解作为搜索方向，并且得到新的迭代点：
$$x_{k+1} = x_{k} + a_k p_k$$

其中我们要求步长$a_k$ 满足Wolfe条件。

这样的迭代与牛顿法类似，区别就在于用近似的Hesse矩阵$B_k$ 代替真实的Hesse矩阵。所以拟牛顿法最关键的地方就是每一步迭代中矩阵$B_k$ 的更新。

现在假设得到一个新的迭代$x_{k+1}$，并得到一个新的二次模型：
<img src='./images/quasi_newton2.png' width='70%'/>

我们尽可能地利用上一步的信息来选取$B_k$。

具体地，我们要求  
$$\nabla f(x_{k+1} - \nabla f(x_k) = a_k B_{k+1}p_k$$

从而得到
$$B_{k+1}(x_{k+1}-x_k) = \nabla f(x_{k+1})-\nabla f(x_k)$$

这个公式被称为割线方程。常用的拟牛顿法有DFP算法和BFGS算法。

---